In [1]:
import pandas as pd
import pymongo
import pprint
from collections import Counter
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [2]:
from pymongo import MongoClient
client = MongoClient()

In [3]:
db = client.msbd5001

### Load Training Data

In [4]:
trData = db.tr_data

In [5]:
trData = trData.find({})
trData = list(trData)
df_trData = pd.DataFrame(trData)
df_trData.head()

,_id,article_contentid,article_dwelltime,article_l1_categories,article_l2_categories,article_lan,article_publisher,article_title,article_update_time,dt,...,user_country,user_gender,user_gender_weight,user_gp_frequency,user_gp_keywords,user_id,user_maxmind_city,user_maxmind_country_iso_code,user_maxmind_state_iso_code,user_model
0,5a0c46d302c885ab9a82a488,1l5c17e72og_us,NaN,NaN,NaN,en,NaN,NaN,NaN,20170111,...,US,2.0,0.763295,"{'COMMUNICATION': 1, 'TOOLS': 5, 'ENTERTAINMEN...",NaN,614E304DE3B07EBCDD4E11382578EF5A,Glen Allen,US,VA,SPH-L710
1,5a0c46d302c885ab9a82a489,mQ56173a2KT_us,NaN,NaN,NaN,en,NaN,NaN,NaN,20170117,...,US,2.0,0.763295,"{'COMMUNICATION': 1, 'TOOLS': 5, 'ENTERTAINMEN...",NaN,614E304DE3B07EBCDD4E11382578EF5A,Glen Allen,US,VA,SPH-L710
2,5a0c46d302c885ab9a82a491,Bm702d2afz8_us,132.0,"[{'weight': 0.47897132942480036, 'name': '1000...","[{'weight': 0.883356596807302, 'name': '100064...",en_US,Buzz Nick,What Was Discovered In The Ocean Is Absolutely...,1.484198e+09,20170112,...,US,2.0,0.539806,"{'TOOLS': 3, 'MUSIC_AND_AUDIO': 2, 'GAME_CASUA...",NaN,56F27AD705D8564F7900470AA793DBD2,Concord,US,NC,E6782
3,5a0c46d302c885ab9a82a4d2,nW7c6db65KY_us,6.0,"[{'weight': 0.9448785614035117, 'name': '10006...","[{'weight': 0.6082519226086233, 'name': '10006...",en_US,Reuters,Trump may reinstate secret CIA 'black site' pr...,1.485381e+09,20170126,...,US,2.0,NaN,"{'TOOLS': 1, 'COMMUNICATION': 2, 'SOCIAL': 1, ...",NaN,F9B30E5BDD8009EBA52C1F5518D9E332,Duluth,US,GA,HTC 2PST2
4,5a0c46d302c885ab9a82a53e,sr1008,6.0,NaN,NaN,en,NaN,NaN,NaN,20170118,...,US,NaN,NaN,{'FINANCE': 1},NaN,A2B8669B00452B841DF3C3C7ECBE0E75,Orange,US,CA,VS985 4G


In [6]:
df_trData = df_trData[~df_trData['user_gp_frequency'].isnull()]
df_trData = df_trData[~df_trData['article_title'].isnull()]

In [144]:
df_ua_map = df_trData[['user_id', 'article_contentid']]
df_ua_map.shape

(168276, 2)

In [145]:
df_ua_map = df_ua_map.drop_duplicates(subset = ['user_id', 'article_contentid'], keep='first')
df_ua_map.shape

(139811, 2)

### Prepare User View

In [9]:
df_user = df_trData[['user_id', 'user_age', 'user_gender', 'user_brand', 'user_maxmind_city', 'user_model']]
df_user.head()

,user_id,user_age,user_gender,user_brand,user_maxmind_city,user_model
2,56F27AD705D8564F7900470AA793DBD2,4.0,2.0,kyocera,Concord,E6782
3,F9B30E5BDD8009EBA52C1F5518D9E332,1.0,2.0,htc,Duluth,HTC 2PST2
5,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung,Glen Allen,SPH-L710
6,CD2F2B61135DB27BA3230F91466ED793,3.0,2.0,samsung,Lebanon,SAMSUNG-SM-G890A
7,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung,Glen Allen,SPH-L710


In [10]:
df_user['user_age'] = df_user['user_age'].fillna(0)
df_user['user_gender'] = df_user['user_gender'].fillna(0)
df_user['user_brand'] = df_user['user_brand'].str.lower().fillna('na')
df_user['user_maxmind_city'] = df_user['user_maxmind_city'].str.lower().fillna('na')
df_user['user_model'] = df_user['user_model'].str.lower().fillna('na')
df_user.head()

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pa

,user_id,user_age,user_gender,user_brand,user_maxmind_city,user_model
2,56F27AD705D8564F7900470AA793DBD2,4.0,2.0,kyocera,concord,e6782
3,F9B30E5BDD8009EBA52C1F5518D9E332,1.0,2.0,htc,duluth,htc 2pst2
5,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung,glen allen,sph-l710
6,CD2F2B61135DB27BA3230F91466ED793,3.0,2.0,samsung,lebanon,samsung-sm-g890a
7,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung,glen allen,sph-l710


In [11]:
df_user = pd.get_dummies(df_user, columns=['user_age', 'user_gender','user_brand', 'user_maxmind_city', 'user_model'])
df_user = df_user.fillna(0)
df_user = df_user.drop_duplicates(subset=['user_id'], keep='first')
df_user.head()

,user_id,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,...,user_model_z955l,user_model_z958,user_model_z959,user_model_z963u,user_model_z963vl,user_model_z970,user_model_z981,user_model_z987,user_model_z988,user_model_zte_n9511
2,56F27AD705D8564F7900470AA793DBD2,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,F9B30E5BDD8009EBA52C1F5518D9E332,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,614E304DE3B07EBCDD4E11382578EF5A,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,CD2F2B61135DB27BA3230F91466ED793,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
15,8EE461BA3AA2595AC91FCDD833EC1711,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
df_user.shape

(7857, 3010)

### Prepare App View

In [13]:
df_app = df_trData[['user_id', 'user_gp_frequency']]
df_app.head()

,user_id,user_gp_frequency
2,56F27AD705D8564F7900470AA793DBD2,"{'TOOLS': 3, 'MUSIC_AND_AUDIO': 2, 'GAME_CASUA..."
3,F9B30E5BDD8009EBA52C1F5518D9E332,"{'TOOLS': 1, 'COMMUNICATION': 2, 'SOCIAL': 1, ..."
5,614E304DE3B07EBCDD4E11382578EF5A,"{'COMMUNICATION': 1, 'TOOLS': 5, 'ENTERTAINMEN..."
6,CD2F2B61135DB27BA3230F91466ED793,"{'COMMUNICATION': 1, 'TOOLS': 3, 'PHOTOGRAPHY'..."
7,614E304DE3B07EBCDD4E11382578EF5A,"{'COMMUNICATION': 1, 'TOOLS': 5, 'ENTERTAINMEN..."


In [14]:
df_app_category = pd.DataFrame.from_records(df_app['user_gp_frequency'].tolist())
df_app_category = df_app_category.fillna(0.1)#change from fillna(0) to fillna(0.1)
df_app_category.head()

,ART_AND_DESIGN,AUTO_AND_VEHICLES,BEAUTY,BOOKS_AND_REFERENCE,BUSINESS,COMICS,COMMUNICATION,DATING,EDUCATION,ENTERTAINMENT,...,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
0,0.1,0.1,0.1,0.1,0.1,0.1,1.0,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,3.0,0.1,0.1,0.1,0.1
1,0.1,0.1,0.1,0.1,0.1,0.1,2.0,0.1,0.1,0.1,...,0.1,0.1,0.1,1.0,0.1,1.0,0.1,0.1,0.1,0.1
2,0.1,0.1,0.1,1.0,1.0,0.1,1.0,0.1,0.1,2.0,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
3,0.1,0.1,0.1,0.1,0.1,0.1,1.0,0.1,0.1,0.1,...,2.0,1.0,0.1,1.0,0.1,3.0,0.1,1.0,0.1,0.1
4,0.1,0.1,0.1,1.0,1.0,0.1,1.0,0.1,0.1,2.0,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1


In [15]:
df_app = df_app.merge(df_app_category, how='left', left_index=True, right_index=True)
df_app = df_app.drop(['user_gp_frequency'], axis=1)
df_app = df_app.drop_duplicates(subset=['user_id'], keep='first')
df_app.head()

,user_id,ART_AND_DESIGN,AUTO_AND_VEHICLES,BEAUTY,BOOKS_AND_REFERENCE,BUSINESS,COMICS,COMMUNICATION,DATING,EDUCATION,...,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
2,56F27AD705D8564F7900470AA793DBD2,0.1,0.1,0.1,1.0,1.0,0.1,1.0,0.1,0.1,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
3,F9B30E5BDD8009EBA52C1F5518D9E332,0.1,0.1,0.1,0.1,0.1,0.1,1.0,0.1,0.1,...,2.0,1.0,0.1,1.0,0.1,3.0,0.1,1.0,0.1,0.1
5,614E304DE3B07EBCDD4E11382578EF5A,0.1,0.1,0.1,1.0,1.0,0.1,1.0,0.1,0.1,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
6,CD2F2B61135DB27BA3230F91466ED793,0.1,0.1,0.1,1.0,1.0,0.1,1.0,0.1,0.1,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
15,8EE461BA3AA2595AC91FCDD833EC1711,0.1,0.1,0.1,0.1,2.0,0.1,4.0,0.1,0.1,...,0.1,0.1,1.0,3.0,0.1,10.0,0.1,3.0,7.0,2.0


In [16]:
df_app = df_app.fillna(0.1)#change from fillna(0) to fillna(0.1)

In [17]:
df_app.shape

(7857, 53)

In [18]:
def sumCat(row):
    l = row.tolist()
    #print l
    row['totalCat'] = sum(l[1:])
    return row

In [19]:
df_app1 = df_app.drop(['user_id'], axis = 1).apply(lambda row: sumCat(row), axis = 1)
df_app1 = df_app[['user_id']].merge(df_app1, how = 'left', left_index=True, right_index=True)
df_app2 = df_app1[df_app1['totalCat'] != 0].drop(['totalCat'], axis = 1)

In [20]:
df_app2.shape

(7857, 53)

In [21]:
df_app2.head()

,user_id,ART_AND_DESIGN,AUTO_AND_VEHICLES,BEAUTY,BOOKS_AND_REFERENCE,BUSINESS,COMICS,COMMUNICATION,DATING,EDUCATION,...,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
2,56F27AD705D8564F7900470AA793DBD2,0.1,0.1,0.1,1.0,1.0,0.1,1.0,0.1,0.1,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
3,F9B30E5BDD8009EBA52C1F5518D9E332,0.1,0.1,0.1,0.1,0.1,0.1,1.0,0.1,0.1,...,2.0,1.0,0.1,1.0,0.1,3.0,0.1,1.0,0.1,0.1
5,614E304DE3B07EBCDD4E11382578EF5A,0.1,0.1,0.1,1.0,1.0,0.1,1.0,0.1,0.1,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
6,CD2F2B61135DB27BA3230F91466ED793,0.1,0.1,0.1,1.0,1.0,0.1,1.0,0.1,0.1,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
15,8EE461BA3AA2595AC91FCDD833EC1711,0.1,0.1,0.1,0.1,2.0,0.1,4.0,0.1,0.1,...,0.1,0.1,1.0,3.0,0.1,10.0,0.1,3.0,7.0,2.0


In [22]:
app_category = df_app.drop(['user_id'], axis = 1).columns
app_category

Index(['ART_AND_DESIGN', 'AUTO_AND_VEHICLES', 'BEAUTY', 'BOOKS_AND_REFERENCE',
       'BUSINESS', 'COMICS', 'COMMUNICATION', 'DATING', 'EDUCATION',
       'ENTERTAINMENT', 'EVENTS', 'FAMILY_ACTION', 'FINANCE', 'FOOD_AND_DRINK',
       'GAME_ACTION', 'GAME_ADVENTURE', 'GAME_ARCADE', 'GAME_BOARD',
       'GAME_CARD', 'GAME_CASINO', 'GAME_CASUAL', 'GAME_EDUCATIONAL',
       'GAME_MUSIC', 'GAME_PUZZLE', 'GAME_RACING', 'GAME_ROLE_PLAYING',
       'GAME_SIMULATION', 'GAME_SPORTS', 'GAME_STRATEGY', 'GAME_TRIVIA',
       'GAME_WORD', 'HEALTH_AND_FITNESS', 'HOUSE_AND_HOME',
       'LIBRARIES_AND_DEMO', 'LIFESTYLE', 'MAPS_AND_NAVIGATION',
       'MEDIA_AND_VIDEO', 'MEDICAL', 'MUSIC_AND_AUDIO', 'NEWS_AND_MAGAZINES',
       'PARENTING', 'PERSONALIZATION', 'PHOTOGRAPHY', 'PRODUCTIVITY',
       'SHOPPING', 'SOCIAL', 'SPORTS', 'TOOLS', 'TRANSPORTATION',
       'TRAVEL_AND_LOCAL', 'VIDEO_PLAYERS', 'WEATHER'],
      dtype='object')

## Merge user and app

In [23]:
df_user_app = df_user.merge(df_app2, how = 'left', on = 'user_id')
#df_user_app = df_user_app.drop(['totalCat'], axis = 1)
df_user_app = df_user_app.fillna(0.1) #change from fillna(0) to fillna(0.1)
df_user_app.head()

,user_id,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,...,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
0,56F27AD705D8564F7900470AA793DBD2,0,0,0,0,1,0,0,1,0,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
1,F9B30E5BDD8009EBA52C1F5518D9E332,0,1,0,0,0,0,0,1,0,...,2.0,1.0,0.1,1.0,0.1,3.0,0.1,1.0,0.1,0.1
2,614E304DE3B07EBCDD4E11382578EF5A,0,0,1,0,0,0,0,1,0,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
3,CD2F2B61135DB27BA3230F91466ED793,0,0,0,1,0,0,0,1,0,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
4,8EE461BA3AA2595AC91FCDD833EC1711,0,0,1,0,0,0,0,1,0,...,0.1,0.1,1.0,3.0,0.1,10.0,0.1,3.0,7.0,2.0


## Get user related category as label

get a table to map user_id with read article and the l1_categories and l2_categories

In [25]:
df_user_app_label = df_trData[['user_id', 'article_contentid', 'article_l1_categories', 'article_l2_categories']]#sophia remark : should add article_update_time as a feature
df_user_app_label['article_l1_categories'] = df_user_app_label['article_l1_categories'].fillna({})
df_user_app_label['article_l2_categories'] = df_user_app_label['article_l2_categories'].fillna({})
df_user_app_label.head()

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,user_id,article_contentid,article_l1_categories,article_l2_categories
2,56F27AD705D8564F7900470AA793DBD2,Bm702d2afz8_us,"[{'weight': 0.47897132942480036, 'name': '1000...","[{'weight': 0.883356596807302, 'name': '100064..."
3,F9B30E5BDD8009EBA52C1F5518D9E332,nW7c6db65KY_us,"[{'weight': 0.9448785614035117, 'name': '10006...","[{'weight': 0.6082519226086233, 'name': '10006..."
5,614E304DE3B07EBCDD4E11382578EF5A,AY628126a1w_us,"[{'weight': 0.9748942874821223, 'name': '10002...","[{'weight': 0.44672383555962764, 'name': '1000..."
6,CD2F2B61135DB27BA3230F91466ED793,jQ6c581bbr2_us,"[{'weight': 0.41806647181510925, 'name': '1001...","[{'weight': 0.41806647181510925, 'name': '1001..."
7,614E304DE3B07EBCDD4E11382578EF5A,1V70d401fdc_us,"[{'weight': 0.8209544133434578, 'name': '10000...","[{'weight': 0.46378997654296183, 'name': '1000..."


Extract l1_category and l2_category from the list

In [26]:
def getDetail(cat, field):
    if (str(cat) != 'nan'):
        if (len(cat) > 0):
            return cat[0][field]
    else:
        return '0'

In [27]:
df_user_app_label['l1_cat'] = df_user_app_label.apply(lambda row: getDetail(row['article_l1_categories'],'name'), axis=1)
df_user_app_label['l2_cat'] = df_user_app_label.apply(lambda row: getDetail(row['article_l2_categories'],'name'), axis=1)
#df_user_app_label['l1_cat_w'] = df_user_app_label.apply(lambda row: getDetail(row['article_l1_categories'],'weight'), axis=1)
#df_user_app_label['l2_cat_w'] = df_user_app_label.apply(lambda row: getDetail(row['article_l2_categories'],'weight'), axis=1)
df_user_app_label = df_user_app_label.drop(['article_l1_categories', 'article_l2_categories'], axis=1)
df_user_app_label.head()

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,user_id,article_contentid,l1_cat,l2_cat
2,56F27AD705D8564F7900470AA793DBD2,Bm702d2afz8_us,1000637,1000644
3,F9B30E5BDD8009EBA52C1F5518D9E332,nW7c6db65KY_us,1000661,1000681
5,614E304DE3B07EBCDD4E11382578EF5A,AY628126a1w_us,1000288,1000288
6,CD2F2B61135DB27BA3230F91466ED793,jQ6c581bbr2_us,1001091,1001091
7,614E304DE3B07EBCDD4E11382578EF5A,1V70d401fdc_us,1000031,1000069


In [28]:
def sumCat(l):
    l = l.unique()
    return l.tolist()

In [29]:
df_user_cat = df_user_app_label.groupby(['user_id']).apply(lambda row: sumCat(row['l2_cat']))
df_user_cat = pd.DataFrame(df_user_cat).reset_index()
df_user_cat = df_user_cat.rename(columns={0:'l2_cat'})
df_user_cat.head()

,user_id,l2_cat
0,00108FC5F1F3031ADD536548124A4912,[1000069]
1,00310455B7B24D5CDEC892351B8D1D51,"[1000395, 1000667, 1000780, 1000611]"
2,00381B68ACD6CC83635CDDD9B40F0BAF,"[1000680, 1000681, 1000671]"
3,003B2215B070CDB50C0819C72E208F0A,"[1000395, 1000288, 1000290, 1000611, 1000069]"
4,003B8D414199B9C5EB7CF8C94BCC6787,[1000069]


In [30]:
def splitCat(row):
    for i in row['l2_cat']:
        if i is not None:
            row['user_l2_cat' + i] = 1
    return row

In [31]:
df_user_cat = df_user_cat.apply(lambda row:splitCat(row), axis = 1)
df_user_cat.head()

,l2_cat,user_id,user_l2_cat0010000000,user_l2_cat1000001,user_l2_cat1000002,user_l2_cat1000007,user_l2_cat10000101,user_l2_cat1000012,user_l2_cat10000162,user_l2_cat1000017,...,user_l2_cat1001040,user_l2_cat1001049,user_l2_cat1001060,user_l2_cat1001091,user_l2_cat1001103,user_l2_cat1001110,user_l2_cat1001117,user_l2_cat2000000,user_l2_cat6000007,user_l2_cat8000001
0,[1000069],00108FC5F1F3031ADD536548124A4912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[1000395, 1000667, 1000780, 1000611]",00310455B7B24D5CDEC892351B8D1D51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[1000680, 1000681, 1000671]",00381B68ACD6CC83635CDDD9B40F0BAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[1000395, 1000288, 1000290, 1000611, 1000069]",003B2215B070CDB50C0819C72E208F0A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[1000069],003B8D414199B9C5EB7CF8C94BCC6787,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df_user_cat = df_user_cat.fillna(0.1).drop(['l2_cat'],axis = 1)#change from fillna(0) to fillna(0.1)
df_user_cat.head()

,user_id,user_l2_cat0010000000,user_l2_cat1000001,user_l2_cat1000002,user_l2_cat1000007,user_l2_cat10000101,user_l2_cat1000012,user_l2_cat10000162,user_l2_cat1000017,user_l2_cat10000177,...,user_l2_cat1001040,user_l2_cat1001049,user_l2_cat1001060,user_l2_cat1001091,user_l2_cat1001103,user_l2_cat1001110,user_l2_cat1001117,user_l2_cat2000000,user_l2_cat6000007,user_l2_cat8000001
0,00108FC5F1F3031ADD536548124A4912,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
1,00310455B7B24D5CDEC892351B8D1D51,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
2,00381B68ACD6CC83635CDDD9B40F0BAF,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
3,003B2215B070CDB50C0819C72E208F0A,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
4,003B8D414199B9C5EB7CF8C94BCC6787,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


In [33]:
#sophia 20171201

In [34]:
df_user_app_cat = df_user_app.merge(df_user_cat, how = 'left', on = 'user_id')
df_user_app_cat.head()

,user_id,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,...,user_l2_cat1001040,user_l2_cat1001049,user_l2_cat1001060,user_l2_cat1001091,user_l2_cat1001103,user_l2_cat1001110,user_l2_cat1001117,user_l2_cat2000000,user_l2_cat6000007,user_l2_cat8000001
0,56F27AD705D8564F7900470AA793DBD2,0,0,0,0,1,0,0,1,0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
1,F9B30E5BDD8009EBA52C1F5518D9E332,0,1,0,0,0,0,0,1,0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
2,614E304DE3B07EBCDD4E11382578EF5A,0,0,1,0,0,0,0,1,0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
3,CD2F2B61135DB27BA3230F91466ED793,0,0,0,1,0,0,0,1,0,...,0.1,0.1,0.1,1.0,0.1,0.1,0.1,0.1,0.1,0.1
4,8EE461BA3AA2595AC91FCDD833EC1711,0,0,1,0,0,0,0,1,0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


### Prepare Article View

In [35]:
df_article = df_trData[['user_id', 'article_contentid', 'article_l1_categories', 'article_l2_categories', 'article_title', 'article_update_time']]#sophia remark : should add article_update_time as a feature
df_article = df_article.sort_values(by=['article_title'], ascending=False)
df_article.head()

,user_id,article_contentid,article_l1_categories,article_l2_categories,article_title,article_update_time
73696,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,"[{'weight': 0.8786688469394612, 'name': '10007...","[{'weight': 0.4648142188685498, 'name': '10006...",﻿Vulnerable teen 'livestreams suicide' on Face...,1.485441e+09
45901,986759E2FD77D818BEDC17BFF6376900,v77c577e1qo_us,"[{'weight': 0.8786688469394612, 'name': '10007...","[{'weight': 0.4648142188685498, 'name': '10006...",﻿Vulnerable teen 'livestreams suicide' on Face...,1.485441e+09
125124,A4F138114721E738AD8C2768C11450A9,v77c577e1qo_us,"[{'weight': 0.8786688469394612, 'name': '10007...","[{'weight': 0.4648142188685498, 'name': '10006...",﻿Vulnerable teen 'livestreams suicide' on Face...,1.485388e+09
136071,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,"[{'weight': 0.04053368419408798, 'name': '1000...","[{'weight': 0.04053368419408798, 'name': '1000...",【E-girls】ちび丸子ちゃん―踊りを学びましょう～,1.482883e+09
136053,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,"[{'weight': 0.04053368419408798, 'name': '1000...","[{'weight': 0.04053368419408798, 'name': '1000...",【E-girls】ちび丸子ちゃん―踊りを学びましょう～,1.482883e+09


In [36]:
from sklearn import preprocessing
#Create x, where x the 'scores' column's values as floats
x = df_article['article_update_time'].values.astype(float)
x = x.reshape(-1,1)
# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x)

# Run the normalizer on the dataframe
df_article['article_update_time'] = x_scaled

In [37]:
df_article = df_article.drop_duplicates(subset=['article_contentid'], keep='first')
df_article = df_article.reset_index(drop = True)
df_article.head()

,user_id,article_contentid,article_l1_categories,article_l2_categories,article_title,article_update_time
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,"[{'weight': 0.8786688469394612, 'name': '10007...","[{'weight': 0.4648142188685498, 'name': '10006...",﻿Vulnerable teen 'livestreams suicide' on Face...,0.950863
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,"[{'weight': 0.04053368419408798, 'name': '1000...","[{'weight': 0.04053368419408798, 'name': '1000...",【E-girls】ちび丸子ちゃん―踊りを学びましょう～,0.342006
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,"[{'weight': 0.999, 'name': '8000001'}]","[{'weight': 0.999, 'name': '8000001'}]",♠ Funny Videos ♠ Best FAILS ♠ December 2016 Co...,0.597340
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,"[{'weight': 0.12680570781230927, 'name': '1000...","[{'weight': 0.12680570781230927, 'name': '1000...",▶[Future Bass] ★ MRVLZ - On The Inside,0.923283
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,"[{'weight': 0.48590736582547495, 'name': '1000...","[{'weight': 0.48590736582547495, 'name': '1000...",€1m in funding sought for new air ambulance se...,0.949038


In [38]:
def getDetail(cat, field):
    if (str(cat) != 'nan'):
        if (len(cat) > 0):
            return cat[0][field]
    else:
        return '0'

In [39]:
df_article['l1_cat'] = df_article.apply(lambda row: getDetail(row['article_l1_categories'],'name'), axis=1)
df_article['l2_cat'] = df_article.apply(lambda row: getDetail(row['article_l2_categories'],'name'), axis=1)
df_article['l1_cat_w'] = df_article.apply(lambda row: getDetail(row['article_l1_categories'],'weight'), axis=1)
df_article['l2_cat_w'] = df_article.apply(lambda row: getDetail(row['article_l2_categories'],'weight'), axis=1)
df_article = df_article.drop(['article_l1_categories', 'article_l2_categories'], axis=1)
df_article.head()

,user_id,article_contentid,article_title,article_update_time,l1_cat,l2_cat,l1_cat_w,l2_cat_w
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,﻿Vulnerable teen 'livestreams suicide' on Face...,0.950863,1000780,1000667,0.878669,0.464814
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,【E-girls】ちび丸子ちゃん―踊りを学びましょう～,0.342006,1000780,1000780,0.040534,0.040534
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,♠ Funny Videos ♠ Best FAILS ♠ December 2016 Co...,0.597340,8000001,8000001,0.999000,0.999000
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,▶[Future Bass] ★ MRVLZ - On The Inside,0.923283,1000931,1000931,0.126806,0.126806
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,€1m in funding sought for new air ambulance se...,0.949038,1000780,1000780,0.485907,0.485907


In [40]:
df_article.shape

(49149, 8)

### Clean title

In [41]:
#df_article = df_article[~df_article['article_title'].isnull()]
#df_article.head()

In [42]:
#skip

from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import re


In [43]:
def hashingTitle(title):
    STEMMER = LancasterStemmer()
    if (type(title) != float) & (type(title) != int):
        title = title.lower()
        title = word_tokenize(title)
        title = [STEMMER.stem(word.strip('!"#$%&\()*+,-./:;<=>?@[\]^_`{|}~').lower()) for word in title]
        titleList = []
        for w in title:
            if len(re.findall("[a-zA-Z]+", w)) > 0:
                f_w = re.findall("[a-zA-Z]+", w)[0].lower()
                f_w = '#'+f_w+'#'
                f_w_tri = [f_w[i:i+3] for i in range(len(f_w)-1) if (len(f_w[i:i+3]) == 3)]
                titleList.extend(f_w_tri)
        return dict(Counter(titleList))
    else:
        return {}

In [44]:
df_article['clean_article_title'] = df_article.apply(lambda x: hashingTitle(x['article_title']), axis=1)
df_article.head()

,user_id,article_contentid,article_title,article_update_time,l1_cat,l2_cat,l1_cat_w,l2_cat_w,clean_article_title
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,﻿Vulnerable teen 'livestreams suicide' on Face...,0.950863,1000780,1000667,0.878669,0.464814,"{'ok#': 1, 'ceb': 1, 'rom': 1, '#te': 1, 'ebo'..."
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,【E-girls】ちび丸子ちゃん―踊りを学びましょう～,0.342006,1000780,1000780,0.040534,0.040534,{'#e#': 1}
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,♠ Funny Videos ♠ Best FAILS ♠ December 2016 Co...,0.597340,8000001,8000001,0.999000,0.999000,"{'il#': 3, 'eo#': 1, 'dec': 1, 'nfa': 1, '#de'..."
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,▶[Future Bass] ★ MRVLZ - On The Inside,0.923283,1000931,1000931,0.126806,0.126806,"{'nsi': 1, '#mr': 1, '#th': 1, 'vlz': 1, 'on#'..."
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,€1m in funding sought for new air ambulance se...,0.949038,1000780,1000780,0.485907,0.485907,"{'ugh': 1, '#so': 1, 'mb#': 1, 'in#': 1, 'ew#'..."


In [45]:
df_article_title = pd.DataFrame.from_records(df_article['clean_article_title'].tolist())
df_article_title = df_article_title.fillna(0)
df_article_title.head()

,#a#,#aa,#ab,#ac,#ad,#ae,#af,#ag,#ah,#ai,...,zyt,zz#,zza,zze,zzf,zzi,zzl,zzn,zzo,zzy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
df_article = df_article.merge(df_article_title, how='left', left_index=True, right_index=True)
df_article.head()

,user_id,article_contentid,article_title,article_update_time,l1_cat,l2_cat,l1_cat_w,l2_cat_w,clean_article_title,#a#,...,zyt,zz#,zza,zze,zzf,zzi,zzl,zzn,zzo,zzy
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,﻿Vulnerable teen 'livestreams suicide' on Face...,0.950863,1000780,1000667,0.878669,0.464814,"{'ok#': 1, 'ceb': 1, 'rom': 1, '#te': 1, 'ebo'...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,【E-girls】ちび丸子ちゃん―踊りを学びましょう～,0.342006,1000780,1000780,0.040534,0.040534,{'#e#': 1},0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,♠ Funny Videos ♠ Best FAILS ♠ December 2016 Co...,0.597340,8000001,8000001,0.999000,0.999000,"{'il#': 3, 'eo#': 1, 'dec': 1, 'nfa': 1, '#de'...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,▶[Future Bass] ★ MRVLZ - On The Inside,0.923283,1000931,1000931,0.126806,0.126806,"{'nsi': 1, '#mr': 1, '#th': 1, 'vlz': 1, 'on#'...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,€1m in funding sought for new air ambulance se...,0.949038,1000780,1000780,0.485907,0.485907,"{'ugh': 1, '#so': 1, 'mb#': 1, 'in#': 1, 'ew#'...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
df_article = df_article.drop(['article_title','clean_article_title'], axis=1)
#Sophia commend : the following line is to be run if title is not included
#df_article = df_article.drop(['article_title'], axis=1)
df_article.head()

,user_id,article_contentid,article_update_time,l1_cat,l2_cat,l1_cat_w,l2_cat_w,#a#,#aa,#ab,...,zyt,zz#,zza,zze,zzf,zzi,zzl,zzn,zzo,zzy
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,0.950863,1000780,1000667,0.878669,0.464814,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,0.342006,1000780,1000780,0.040534,0.040534,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,0.597340,8000001,8000001,0.999000,0.999000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,0.923283,1000931,1000931,0.126806,0.126806,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,0.949038,1000780,1000780,0.485907,0.485907,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
#df_article = pd.get_dummies(df_article, columns=['article_publisher'])
df_article = df_article.fillna(0)
df_article.head()

,user_id,article_contentid,article_update_time,l1_cat,l2_cat,l1_cat_w,l2_cat_w,#a#,#aa,#ab,...,zyt,zz#,zza,zze,zzf,zzi,zzl,zzn,zzo,zzy
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,0.950863,1000780,1000667,0.878669,0.464814,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,0.342006,1000780,1000780,0.040534,0.040534,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,0.597340,8000001,8000001,0.999000,0.999000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,0.923283,1000931,1000931,0.126806,0.126806,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,0.949038,1000780,1000780,0.485907,0.485907,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
df_cat = df_article[['l1_cat', 'l2_cat', 'l1_cat_w', 'l2_cat_w']]
df_cat = pd.get_dummies(df_cat, columns=['l1_cat', 'l2_cat'])
df_cat = df_cat.fillna(0.1)
df_cat.head()

,l1_cat_w,l2_cat_w,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,0.878669,0.464814,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.040534,0.040534,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.999000,0.999000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0.126806,0.126806,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.485907,0.485907,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
df_cat = df_cat.merge(df_article[['l1_cat', 'l2_cat']], how='left', left_index=True, right_index=True)
df_cat.head()

,l1_cat_w,l2_cat_w,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,...,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001,l1_cat,l2_cat
0,0.878669,0.464814,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000780,1000667
1,0.040534,0.040534,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000780,1000780
2,0.999000,0.999000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,8000001,8000001
3,0.126806,0.126806,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000931,1000931
4,0.485907,0.485907,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000780,1000780


In [51]:
def updateCatWeight(row):
    cat = row['l1_cat']
    row['l1_cat_'+str(cat)] = row['l1_cat_w']
    return row

In [52]:
df_cat = df_cat.apply(lambda row: updateCatWeight(row), axis=1)
df_cat.head()

,l1_cat_w,l2_cat_w,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,...,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001,l1_cat,l2_cat
0,0.878669,0.464814,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1000780,1000667
1,0.040534,0.040534,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1000780,1000780
2,0.999000,0.999000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,8000001,8000001
3,0.126806,0.126806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1000931,1000931
4,0.485907,0.485907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1000780,1000780


In [53]:
df_cat = df_cat.drop(['l1_cat','l2_cat','l1_cat_w', 'l2_cat_w'], axis=1)
df_cat.head()

,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,l1_cat_1000288,l1_cat_1000298,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
df_article = df_article.merge(df_cat, how='left', left_index=True, right_index=True)
#sophia remark: commend here
#df_article = df_article.drop(['article_publisher','article_title'], axis = 1)
df_article.head()

,user_id,article_contentid,article_update_time,l1_cat,l2_cat,l1_cat_w,l2_cat_w,#a#,#aa,#ab,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,0.950863,1000780,1000667,0.878669,0.464814,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,0.342006,1000780,1000780,0.040534,0.040534,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,0.597340,8000001,8000001,0.999000,0.999000,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,0.923283,1000931,1000931,0.126806,0.126806,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,0.949038,1000780,1000780,0.485907,0.485907,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
df_article_2 = df_article.drop(['l1_cat','l2_cat','l1_cat_w','l2_cat_w'], axis = 1)
df_article_2.shape

(49149, 8066)

### Prepare Train Test Set

In [64]:
import random
train_ratio = 0.9
num_train_user = int(round(len(df_user)*train_ratio))
train_user_id = random.sample(df_user['user_id'].tolist(), num_train_user)

In [65]:
len(train_user_id)

7071

In [66]:
test_user_id = df_user[~df_user['user_id'].isin(train_user_id)]['user_id'].tolist()
len(test_user_id)

786

In [67]:
old_new_ratio = 0.8
num_old_user = int(round(len(test_user_id)*train_ratio))
old_user_id = random.sample(test_user_id, num_old_user)
len(old_user_id)

707

In [68]:
new_user_id = [i for i in test_user_id if i not in old_user_id]
len(new_user_id)

79

In [69]:
final_train_user_id = []
final_train_user_id.extend(train_user_id)
final_train_user_id.extend(old_user_id)
len(final_train_user_id)

7778

In [70]:
final_test_user_id = []
final_test_user_id.extend(new_user_id)
final_test_user_id.extend(old_user_id)
len(final_test_user_id)

786

In [71]:
df_train_user_0 = df_user_app[df_user_app['user_id'].isin(final_train_user_id)].fillna(0.1)
df_test_user_0 = df_user_app[df_user_app['user_id'].isin(final_test_user_id)].fillna(0.1)

In [72]:
a = df_train_user_0.drop(['user_id'], axis = 1).as_matrix()
samples = list(a)
neigh = NearestNeighbors(n_neighbors=1)
neigh.fit(samples) 

#get the nearest user index - use for loop later
#print(neigh.kneighbors([samples[15]])) [1][0]

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=1, p=2, radius=1.0)

In [73]:
def findNeighbor(row):
    l = row.values
    row['neibor'] = neigh.kneighbors([l]) [1][0][0]
    return row

In [74]:
df_test_user_1 = df_test_user_0.drop(['user_id'],axis = 1).apply(lambda row: findNeighbor(row), axis = 1)#drop(['user_id'],axis = 1)
df_test_user_1.head()

,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,user_brand_asus,...,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER,neibor
6,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.1,2.0,0.1,3.0,0.1,1.0,1.0,0.1,6
8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,4.0,0.1,2.0,0.1,16.0,0.1,1.0,1.0,0.1,8
11,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,7.0,0.1,5.0,0.1,5.0,0.1,1.0,2.0,3.0,11
20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.1,2.0,0.1,0.1,0.1,0.1,0.1,1.0,0.1,20
44,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,6.0,1.0,8.0,1.0,16.0,0.1,2.0,6.0,1.0,43


In [75]:
df_user_cat.head()

,user_id,user_l2_cat0010000000,user_l2_cat1000001,user_l2_cat1000002,user_l2_cat1000007,user_l2_cat10000101,user_l2_cat1000012,user_l2_cat10000162,user_l2_cat1000017,user_l2_cat10000177,...,user_l2_cat1001040,user_l2_cat1001049,user_l2_cat1001060,user_l2_cat1001091,user_l2_cat1001103,user_l2_cat1001110,user_l2_cat1001117,user_l2_cat2000000,user_l2_cat6000007,user_l2_cat8000001
0,00108FC5F1F3031ADD536548124A4912,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
1,00310455B7B24D5CDEC892351B8D1D51,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
2,00381B68ACD6CC83635CDDD9B40F0BAF,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
3,003B2215B070CDB50C0819C72E208F0A,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
4,003B8D414199B9C5EB7CF8C94BCC6787,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


In [76]:
df_train_user = df_train_user_0.merge(df_user_cat, how = 'left', on = 'user_id')

In [77]:
art_cat_col = [i for i in df_train_user.columns if i not in df_train_user_0.columns]

In [78]:
user_fea_col = [i for i in df_train_user.columns \
                if i not in app_category and 'user_maxmind_city' not in i\
                and 'user_brand' not in i  and 'user_id' != i]
len(user_fea_col)

812

In [79]:
test_col = [i for i in df_train_user.columns \
                if i not in app_category and 'user_maxmind_city' not in i]

In [80]:
df_test_user = df_test_user_1.merge(df_train_user[art_cat_col], how = 'left', left_on='neibor', right_index=True)
df_test_user = df_test_user.drop(['neibor'],axis = 1)
df_test_user.head()

,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,user_brand_asus,...,user_l2_cat1001040,user_l2_cat1001049,user_l2_cat1001060,user_l2_cat1001091,user_l2_cat1001103,user_l2_cat1001110,user_l2_cat1001117,user_l2_cat2000000,user_l2_cat6000007,user_l2_cat8000001
6,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
11,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
44,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.1,0.1,0.1,1.0,0.1,0.1,0.1,0.1,1.0,0.1


In [81]:
df_test_user.shape, df_train_user.shape

((786, 3216), (7778, 3217))

In [84]:
user_fea_col

['user_age_0.0',
 'user_age_1.0',
 'user_age_2.0',
 'user_age_3.0',
 'user_age_4.0',
 'user_gender_0.0',
 'user_gender_1.0',
 'user_gender_2.0',
 'user_model_0paj5',
 'user_model_0pcv1',
 'user_model_0pja10',
 'user_model_0pja2',
 'user_model_0pm92',
 'user_model_2pq93',
 'user_model_2ps64',
 'user_model_2pyb2',
 'user_model_306sh',
 'user_model_5017b',
 'user_model_5027b',
 'user_model_5042t',
 'user_model_5054n',
 'user_model_5054w',
 'user_model_5056e',
 'user_model_5056n',
 'user_model_5065n',
 'user_model_5098s',
 'user_model_5560s',
 'user_model_6045i',
 'user_model_6045o',
 'user_model_7040n',
 'user_model_7040t',
 'user_model_710c',
 'user_model_801es',
 'user_model_831c',
 'user_model_9006w',
 'user_model_9020a',
 'user_model_a0001',
 'user_model_a1',
 'user_model_a1-810',
 'user_model_a1x plus',
 'user_model_a460g',
 'user_model_a462c',
 'user_model_a463bg',
 'user_model_a464bg',
 'user_model_a521l',
 'user_model_a571vl',
 'user_model_adr6400l',
 'user_model_alcatel a564c',
 

In [85]:
#df_test_user = df_user[df_user['user_id'].isin(final_test_user_id)]
#df_test_user = df_test_user.drop(['user_id'], axis=1)
df_test_user_2 = df_test_user[user_fea_col].fillna(0)
#df_test_user_2 = df_test_user.fillna(0)
df_test_user_arr = df_test_user_2.values
df_test_user_arr.shape

(786, 812)

In [86]:
app_category

Index(['ART_AND_DESIGN', 'AUTO_AND_VEHICLES', 'BEAUTY', 'BOOKS_AND_REFERENCE',
       'BUSINESS', 'COMICS', 'COMMUNICATION', 'DATING', 'EDUCATION',
       'ENTERTAINMENT', 'EVENTS', 'FAMILY_ACTION', 'FINANCE', 'FOOD_AND_DRINK',
       'GAME_ACTION', 'GAME_ADVENTURE', 'GAME_ARCADE', 'GAME_BOARD',
       'GAME_CARD', 'GAME_CASINO', 'GAME_CASUAL', 'GAME_EDUCATIONAL',
       'GAME_MUSIC', 'GAME_PUZZLE', 'GAME_RACING', 'GAME_ROLE_PLAYING',
       'GAME_SIMULATION', 'GAME_SPORTS', 'GAME_STRATEGY', 'GAME_TRIVIA',
       'GAME_WORD', 'HEALTH_AND_FITNESS', 'HOUSE_AND_HOME',
       'LIBRARIES_AND_DEMO', 'LIFESTYLE', 'MAPS_AND_NAVIGATION',
       'MEDIA_AND_VIDEO', 'MEDICAL', 'MUSIC_AND_AUDIO', 'NEWS_AND_MAGAZINES',
       'PARENTING', 'PERSONALIZATION', 'PHOTOGRAPHY', 'PRODUCTIVITY',
       'SHOPPING', 'SOCIAL', 'SPORTS', 'TOOLS', 'TRANSPORTATION',
       'TRAVEL_AND_LOCAL', 'VIDEO_PLAYERS', 'WEATHER'],
      dtype='object')

In [87]:
df_test_app0 = df_app2[df_app2['user_id'].isin(final_test_user_id)]
df_test_app_2 = df_test_user_0[['user_id']].merge(df_test_app0, how = 'left', on = 'user_id')[app_category]
df_test_app_2.head()

,ART_AND_DESIGN,AUTO_AND_VEHICLES,BEAUTY,BOOKS_AND_REFERENCE,BUSINESS,COMICS,COMMUNICATION,DATING,EDUCATION,ENTERTAINMENT,...,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
0,0.1,0.1,0.1,0.1,0.1,0.1,3.0,0.1,0.1,1.0,...,1.0,1.0,0.1,2.0,0.1,3.0,0.1,1.0,1.0,0.1
1,0.1,0.1,0.1,6.0,1.0,0.1,5.0,0.1,2.0,4.0,...,2.0,4.0,0.1,2.0,0.1,16.0,0.1,1.0,1.0,0.1
2,0.1,0.1,0.1,2.0,0.1,0.1,4.0,0.1,4.0,4.0,...,2.0,7.0,0.1,5.0,0.1,5.0,0.1,1.0,2.0,3.0
3,0.1,0.1,0.1,1.0,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,2.0,0.1,0.1,0.1,0.1,0.1,1.0,0.1
4,0.1,0.1,0.1,6.0,0.1,0.1,7.0,0.1,1.0,10.0,...,2.0,6.0,1.0,8.0,1.0,16.0,0.1,2.0,6.0,1.0


In [88]:
df_test_app_2 = df_test_app_2.fillna(0.1)
df_test_user_arr = df_test_app_2.values
df_test_user_arr.shape

(786, 52)

In [89]:
# may need to do sampling on df_ua_map
###sophia 20171201
#df_ua_map = df_ua_map.sample(frac=0.3, replace=False)

In [90]:
len(df_ua_map)

139811

In [148]:
#df_train_user = df_train_user_0.merge(df_user_cat, how = 'left', on = 'user_id')

In [149]:
all_user_id = train_user_id + old_user_id
df_final_train_user_article = df_ua_map[(df_ua_map['user_id'].isin(all_user_id))]
df_final_train_user_article = df_final_train_user_article.merge(df_train_user[test_col], how = 'left', on = 'user_id')

In [150]:
print (df_final_train_user_article.shape)
df_final_train_user_article.head()

(138815, 860)


,user_id,article_contentid,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,...,user_l2_cat1001040,user_l2_cat1001049,user_l2_cat1001060,user_l2_cat1001091,user_l2_cat1001103,user_l2_cat1001110,user_l2_cat1001117,user_l2_cat2000000,user_l2_cat6000007,user_l2_cat8000001
0,56F27AD705D8564F7900470AA793DBD2,Bm702d2afz8_us,0,0,0,0,1,0,0,1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
1,F9B30E5BDD8009EBA52C1F5518D9E332,nW7c6db65KY_us,0,1,0,0,0,0,0,1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
2,614E304DE3B07EBCDD4E11382578EF5A,AY628126a1w_us,0,0,1,0,0,0,0,1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
3,CD2F2B61135DB27BA3230F91466ED793,jQ6c581bbr2_us,0,0,0,1,0,0,0,1,...,0.1,0.1,0.1,1.0,0.1,0.1,0.1,0.1,0.1,0.1
4,614E304DE3B07EBCDD4E11382578EF5A,1V70d401fdc_us,0,0,1,0,0,0,0,1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


In [151]:
df_final_train_user_article = df_final_train_user_article.merge(df_article_2.drop(['user_id'],axis = 1), how = 'left', on = 'article_contentid')
df_final_train_user_article.head()

,user_id,article_contentid,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,56F27AD705D8564F7900470AA793DBD2,Bm702d2afz8_us,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,F9B30E5BDD8009EBA52C1F5518D9E332,nW7c6db65KY_us,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,614E304DE3B07EBCDD4E11382578EF5A,AY628126a1w_us,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,CD2F2B61135DB27BA3230F91466ED793,jQ6c581bbr2_us,0,0,0,1,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4,614E304DE3B07EBCDD4E11382578EF5A,1V70d401fdc_us,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [152]:
df_final_train_user_article.shape

(138815, 8924)

In [153]:
df_final_train_user_article = df_final_train_user_article.iloc[[i for i in df_final_train_user_article.index if i%2 == 0]].reset_index(drop=True)
df_final_train_user_article.head()

,user_id,article_contentid,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,56F27AD705D8564F7900470AA793DBD2,Bm702d2afz8_us,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,614E304DE3B07EBCDD4E11382578EF5A,AY628126a1w_us,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,614E304DE3B07EBCDD4E11382578EF5A,1V70d401fdc_us,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,614E304DE3B07EBCDD4E11382578EF5A,Ly7407d4cuC_us,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,CD2F2B61135DB27BA3230F91466ED793,vm721d1d5Qp_us,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [154]:
#df_final_train = df_final_train_user_article.merge(df_user, how='left', on='user_id')
df_final_train = df_final_train_user_article.fillna(0.1)
#sophia remark: add app data
#df_final_train = df_final_train.merge(df_app2, how='left', on='user_id')

In [155]:
df_final_train.shape

(69408, 8924)

In [156]:
df_final_train = df_final_train.merge(df_app2, how='left', on='user_id')
###sophia remark, check wether there is NAN or all 0 here
#df_final_train = df_final_train.dropna()
#df_final_train.head()

In [157]:
df_final_train.shape

(69408, 8976)

In [158]:
user_feature = [i for i in df_final_train.columns if 'user_' in i and i != 'user_id']
user_feature

['user_age_0.0',
 'user_age_1.0',
 'user_age_2.0',
 'user_age_3.0',
 'user_age_4.0',
 'user_gender_0.0',
 'user_gender_1.0',
 'user_gender_2.0',
 'user_brand_acer',
 'user_brand_asus',
 'user_brand_att',
 'user_brand_blackberry',
 'user_brand_blu',
 'user_brand_cingular_us',
 'user_brand_coolpad',
 'user_brand_cubot',
 'user_brand_dragon touch',
 'user_brand_emerson',
 'user_brand_gionee',
 'user_brand_google',
 'user_brand_hisense',
 'user_brand_honor',
 'user_brand_htc',
 'user_brand_huawei',
 'user_brand_intex',
 'user_brand_kyocera',
 'user_brand_lenovo',
 'user_brand_lge',
 'user_brand_metropcs',
 'user_brand_micromax',
 'user_brand_motorola',
 'user_brand_myphone',
 'user_brand_nextbit',
 'user_brand_nextbook',
 'user_brand_oneplus',
 'user_brand_rca',
 'user_brand_samsung',
 'user_brand_sharp',
 'user_brand_sky devices',
 'user_brand_sony',
 'user_brand_speed',
 'user_brand_tcl',
 'user_brand_tcl_metropcs',
 'user_brand_tct',
 'user_brand_tct_metropcs',
 'user_brand_trio',
 'use

In [159]:
article_feature = [i for i in df_final_train.columns if 'user_' not in i and i not in ['article_contentid',
 'article_title',
 'article_publisher',
 'l1_cat',
 'l2_cat',
 'l1_cat_w',
 'l2_cat_w',
 'l1_cat_w_x',
 'l2_cat_w_x',
 'l1_cat_w_y',
 'l2_cat_w_y',
 'totalCat'] and i not in app_category]
#article_feature = 
article_feature

['article_update_time',
 '#a#',
 '#aa',
 '#ab',
 '#ac',
 '#ad',
 '#ae',
 '#af',
 '#ag',
 '#ah',
 '#ai',
 '#aj',
 '#ak',
 '#al',
 '#am',
 '#an',
 '#ao',
 '#ap',
 '#aq',
 '#ar',
 '#as',
 '#at',
 '#au',
 '#av',
 '#aw',
 '#ax',
 '#ay',
 '#az',
 '#b#',
 '#ba',
 '#bb',
 '#bc',
 '#bd',
 '#be',
 '#bf',
 '#bg',
 '#bh',
 '#bi',
 '#bj',
 '#bk',
 '#bl',
 '#bm',
 '#bn',
 '#bo',
 '#bp',
 '#br',
 '#bs',
 '#bt',
 '#bu',
 '#bv',
 '#bw',
 '#by',
 '#c#',
 '#ca',
 '#cb',
 '#cc',
 '#cd',
 '#ce',
 '#cf',
 '#cg',
 '#ch',
 '#ci',
 '#ck',
 '#cl',
 '#cm',
 '#cn',
 '#co',
 '#cp',
 '#cq',
 '#cr',
 '#cs',
 '#ct',
 '#cu',
 '#cv',
 '#cw',
 '#cx',
 '#cy',
 '#cz',
 '#d#',
 '#da',
 '#db',
 '#dc',
 '#dd',
 '#de',
 '#df',
 '#dg',
 '#dh',
 '#di',
 '#dj',
 '#dk',
 '#dl',
 '#dm',
 '#dn',
 '#do',
 '#dp',
 '#dr',
 '#ds',
 '#dt',
 '#du',
 '#dv',
 '#dw',
 '#dx',
 '#dy',
 '#dz',
 '#e#',
 '#ea',
 '#eb',
 '#ec',
 '#ed',
 '#ee',
 '#ef',
 '#eg',
 '#ei',
 '#ej',
 '#ek',
 '#el',
 '#em',
 '#en',
 '#eo',
 '#ep',
 '#eq',
 '#er',
 '#es',


In [160]:
df_train_user = df_final_train[user_feature]
df_train_user.head()

,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,user_brand_asus,...,user_l2_cat1001040,user_l2_cat1001049,user_l2_cat1001060,user_l2_cat1001091,user_l2_cat1001103,user_l2_cat1001110,user_l2_cat1001117,user_l2_cat2000000,user_l2_cat6000007,user_l2_cat8000001
0,0,0,0,0,1,0,0,1,0,0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
1,0,0,1,0,0,0,0,1,0,0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
2,0,0,1,0,0,0,0,1,0,0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
3,0,0,1,0,0,0,0,1,0,0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
4,0,0,0,1,0,0,0,1,0,0,...,0.1,0.1,0.1,1.0,0.1,0.1,0.1,0.1,0.1,0.1


In [161]:
df_train_app = df_final_train[app_category]
df_train_app.head()

,ART_AND_DESIGN,AUTO_AND_VEHICLES,BEAUTY,BOOKS_AND_REFERENCE,BUSINESS,COMICS,COMMUNICATION,DATING,EDUCATION,ENTERTAINMENT,...,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
0,0.1,0.1,0.1,1.0,1.0,0.1,1.0,0.1,0.1,2.0,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
1,0.1,0.1,0.1,1.0,1.0,0.1,1.0,0.1,0.1,2.0,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
2,0.1,0.1,0.1,1.0,1.0,0.1,1.0,0.1,0.1,2.0,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
3,0.1,0.1,0.1,1.0,1.0,0.1,1.0,0.1,0.1,2.0,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
4,0.1,0.1,0.1,1.0,1.0,0.1,1.0,0.1,0.1,2.0,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1


In [162]:
df_train_article = df_final_train[article_feature]
df_train_article.head()

,article_update_time,#a#,#aa,#ab,#ac,#ad,#ae,#af,#ag,#ah,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,0.654868,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0.221931,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0.710475,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,0.764028,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0.713479,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [163]:
df_train_user.shape

(69408, 858)

In [164]:
df_train_app.shape

(69408, 52)

In [165]:
df_train_article.shape

(69408, 8064)

In [166]:
#df_final_train[df_final_train['user_id'] == 'F4CCDDAA1BAE16B0AF3BCB82801A3CF7']

In [167]:
#df_article[df_article['user_id'] == 'F4CCDDAA1BAE16B0AF3BCB82801A3CF7']

In [168]:
df_test_article = df_article.drop_duplicates(subset=['article_contentid'], keep='first')[article_feature]
df_test_article.shape

(49149, 8064)

In [169]:
len(article_feature)

8064

In [170]:
article_feature.append(u'article_contentid')
len(article_feature)

8065

In [171]:
df_test_article0 = df_article.drop_duplicates(subset=['article_contentid'], keep='first')#[article_feature]
df_test_article0.head()

,user_id,article_contentid,article_update_time,l1_cat,l2_cat,l1_cat_w,l2_cat_w,#a#,#aa,#ab,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,0.950863,1000780,1000667,0.878669,0.464814,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,0.342006,1000780,1000780,0.040534,0.040534,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,0.597340,8000001,8000001,0.999000,0.999000,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,0.923283,1000931,1000931,0.126806,0.126806,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,0.949038,1000780,1000780,0.485907,0.485907,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [172]:
df_test_article_arr1 = df_test_article.values

In [173]:
df_test_article_arr1.shape

(49149, 8064)

In [179]:
y_train = [1 for i in range(len(df_train_user.values))]
len(y_train)

69408

### Build Model By Keras (User Vs App Vs Article)

In [180]:
df_train_user.columns

Index(['user_age_0.0', 'user_age_1.0', 'user_age_2.0', 'user_age_3.0',
       'user_age_4.0', 'user_gender_0.0', 'user_gender_1.0', 'user_gender_2.0',
       'user_brand_acer', 'user_brand_asus',
       ...
       'user_l2_cat1001040', 'user_l2_cat1001049', 'user_l2_cat1001060',
       'user_l2_cat1001091', 'user_l2_cat1001103', 'user_l2_cat1001110',
       'user_l2_cat1001117', 'user_l2_cat2000000', 'user_l2_cat6000007',
       'user_l2_cat8000001'],
      dtype='object', length=858)

In [181]:
df_train_user.shape, df_train_app.shape

((69408, 858), (69408, 52))

In [182]:
df_train_user_arr = df_train_user[user_fea_col].values
df_train_app_arr = df_train_app.values

In [183]:
df_train_user_arr.shape, df_train_app_arr.shape

((69408, 812), (69408, 52))

In [184]:
df_train_article.shape

(69408, 8064)

In [185]:
df_train_article_arr = df_train_article.values
df_train_article_arr.shape

(69408, 8064)

In [186]:
y_train = [1 for i in range(len(df_train_user))]
len(y_train)

69408

In [187]:
from keras.models import Sequential, Model
from keras.layers import Dense, Merge, Activation, Dropout, Reshape

In [188]:
# create user model
userModel1 = Sequential()
userModel1.add(Dense(500, input_shape=(812,)))
userModel1.add(Activation('tanh'))
userModel1.add(Dropout(0.4))
userModel1.add(Dense(50, input_shape=(500,)))
userModel1.add(Activation('tanh'))
userModel1.add(Dropout(0.4))
userModel1.add(Dense(30, input_shape=(50,)))
userModel1.add(Activation('tanh'))
userModel1.add(Dropout(0.4))
userModel1.add(Dense(10, input_shape=(30,)))
userModel1.add(Activation('tanh'))
userModel1.add(Reshape((10,1)))

In [189]:
# create article model
articleModel = Sequential()
articleModel.add(Dense(2000, input_shape=(8064,)))
articleModel.add(Activation('tanh'))
articleModel.add(Dropout(0.4))
articleModel.add(Dense(500, input_shape=(2000,)))
articleModel.add(Activation('tanh'))
articleModel.add(Dropout(0.4))
articleModel.add(Dense(50, input_shape=(500,)))
articleModel.add(Activation('tanh'))
articleModel.add(Dropout(0.4))
articleModel.add(Dense(30, input_shape=(50,)))
articleModel.add(Activation('tanh'))
articleModel.add(Dropout(0.4))
articleModel.add(Dense(10, input_shape=(30,)))
articleModel.add(Activation('tanh'))
articleModel.add(Reshape((10,1)))

In [190]:
model1 = Sequential()
model1.add(Merge([userModel1,articleModel], mode='cos', dot_axes=1, output_shape=(1,)))
model1.add(Reshape((1,)))

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  from ipykernel import kernelapp as app


In [191]:
# create user model 2
userModel2 = Sequential()
userModel2.add(Dense(30, input_shape=(52,)))
userModel2.add(Activation('tanh'))
userModel2.add(Dropout(0.4))
userModel2.add(Dense(10, input_shape=(30,)))
userModel2.add(Activation('tanh'))
userModel2.add(Reshape((10,1)))

In [192]:
model2 = Sequential()
model2.add(Merge([userModel2,articleModel], mode='cos', dot_axes=1, output_shape=(1,)))
model2.add(Reshape((1,)))

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  from ipykernel import kernelapp as app


In [193]:
model=Sequential()
model.add(Merge([model1,model2],mode='concat'))
model.add(Dense(1, input_shape=(2,)))
model.add(Reshape((1,)))
#model.add(Activation('normalization'))
#model.add(Reshape(1,))
#model.add(Activation('softmax'))

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  from ipykernel import kernelapp as app


In [194]:
# Compile model
model.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])

In [195]:
len(df_test_article_arr1)

49149

In [ ]:
model.fit([df_train_user_arr ,df_train_article_arr,df_train_app_arr] ,y_train, epochs=1, batch_size=32)

Epoch 1/1


In [ ]:
# Compile model 1
#model1.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy','precision','recall'])
#model1.fit([df_train_user_arr ,df_train_article_arr] ,y_train, nb_epoch=1, batch_size=32)

# Compile model 2
#model2.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy','precision','recall'])
#model2.fit([df_train_app_arr ,df_train_article_arr] ,y_train, nb_epoch=1, batch_size=32)

In [ ]:
#y_pred = model.predict([df_test_user,df_test_article_arr1, df_test_app], batch_size=32, verbose=0)
#y_pred_1 = np.hstack(y_pred)
#np.argsort(y_pred_1)[:100]

### Calculate precision and recall

In [ ]:
#sophia remark get the mapping of user article
#df_ua_map = df_trData[['user_id', 'article_contentid']]

#Sophia remark: get test user list
#df_test_user0 = df_user[df_user['user_id'].isin(final_test_user_id)]
#df_test_user = df_user[df_user['user_id'].isin(final_test_user_id)]
#df_test_user = df_test_user.drop(['user_id'], axis=1)
#df_test_user = df_test_user.fillna(0)
df_test_user_arr1 = df_test_user[user_fea_col].values
#df_test_user_arr1.shape

#Sophia remark: get test app list
df_test_app0 = df_app2[df_app2['user_id'].isin(final_test_user_id)]
df_test_app = df_user[df_user['user_id'].isin(final_test_user_id)].merge(df_test_app0, how = 'left', on = 'user_id')[app_category]
df_test_app = df_test_app.fillna(0)
df_test_app_arr1 = df_test_app.values
#df_test_app_arr1.shape

#sophia remark: get article list
df_test_article0 = df_article.drop_duplicates(subset=['article_contentid'], keep='first')#[article_feature]
#df_test_article0.head()

df_test_article_arr1 = df_test_article.values

In [ ]:
t_u = df_test_user[user_fea_col]
t_u = t_u.reset_index(drop=True)
t_u.head()

In [ ]:
df_test_user0 = df_user[df_user['user_id'].isin(final_test_user_id)]

In [ ]:
df_test_article_arr1[0:5]

In [ ]:
l = len(df_test_article_arr1)
uid = df_test_user0['user_id'].tolist()[0]
user = np.array([df_test_user_arr1[0]]*l)
user
app = np.array([df_test_app_arr1[0]]*l)
#apply user and article array to do predict
up = pd.DataFrame(model1.predict([user,df_test_article_arr1], batch_size=32))
up.head()

In [ ]:
l = len(df_test_article_arr1)
precision = list()
recall = list()

for idx, user in enumerate(df_test_user_arr1):
    if idx > 0:
        break;
    #for each user-article pair: get the user id and user info
    uid = df_test_user0['user_id'].tolist()[idx]
    user = np.array([df_test_user_arr1[idx]]*l)
    app = np.array([df_test_app_arr1[idx]]*l)
    #apply user and article array to do predict
    up = pd.DataFrame(model.predict([user,df_test_article_arr1, app], batch_size=32))
    df_test_article1=df_test_article0.reset_index(drop = True)\
                            .merge(up, how = 'left', left_index=True, right_index=True)\
                            .rename(columns={0:'u0_score'})[['article_contentid', 'u0_score']]
    df_test_article1 = df_test_article1.sort_values(by='u0_score', ascending=False)
    df_test_article1['y_pred'] = 0
    df_test_article1[:5]['y_pred']=1
    df_match = df_ua_map[df_ua_map['user_id'] == uid][['article_contentid']]
    df_match['y_true'] = 1
    df_test_article1 = df_test_article1.merge(df_match, how = 'left', on = 'article_contentid')\
                                       .fillna(0)
    tp = df_test_article1[(df_test_article1['y_pred'] == 1) & (df_test_article1['y_true'] == 1)].shape[0]
    fp = df_test_article1[(df_test_article1['y_pred'] == 1) & (df_test_article1['y_true'] == 0)].shape[0]
    fn = df_test_article1[(df_test_article1['y_pred'] == 0) & (df_test_article1['y_true'] == 1)].shape[0]

    p = tp/(float(tp+fp))
    r = tp/(float(tp+fn))
    print (idx, uid, p,r)
    if p != 0:
        precision.append(p)
    if r != 0:
        recall.append(r)
    
print (precision, recall)

In [ ]:
df_test_article1.head()

In [ ]:
df_test_article1[df_test_article1['article_contentid'].isin(list(df_trData[df_trData['user_id'] == 'B3B51127D5FB5B39E4273BE65CACA28C']['article_contentid']))]

In [ ]:
df_ua_map[df_ua_map['user_id'] == '5BA72C11B1E94BA50F1E66F4B1362CE6']

In [ ]:
list(df_trData[df_trData['user_id'] == '5BA72C11B1E94BA50F1E66F4B1362CE6']['article_contentid'])

In [ ]:
t_u.head()

In [ ]:
t_u['index'] = t_u.index
t_u_group = t_u.groupby(by=[i for i in t_u.columns if i != 'index']).agg({'index':'nunique'}).reset_index(drop=True)
t_u_group = t_u_group.sort_values(by=['index'], ascending=False)
t_u_group.head()